<a href="https://colab.research.google.com/github/j0gea/Machine-Learning-Sidewalk-damage/blob/main/yolov5_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 구글 드라이브 파일 압축풀기

- s 는 추가파일이라 할 필요 x

In [ ]:
%cd /content/drive/MyDrive/yolov5/DATA/

In [ ]:
!ls

In [ ]:
# 라벨 파일 압축풀기
%cd /content/drive/MyDrive/yolov5/DATA/label_temp

!unzip -qq "/content/drive/MyDrive/yolov5/DATA/

In [ ]:
# 그림 파일 압축풀기
%cd /content/drive/MyDrive/yolov5/DATA/data/images

!unzip -qq "/content/drive/MyDrive/yolov5/DATA/

In [ ]:
# 파일 개수
!ls -l | grep ^- | wc -l

In [ ]:
# 파일 삭제
# !rm -rf ./*.txt
# !rm -rf ./*.json

### YOLOv5 설치

yolov5 설정

In [ ]:
%cd 설치 폴더로 이동

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
%cd yolov5

In [ ]:
! pip install -r requirements.txt

## YOLOv5 Colab
- https://velog.io/@jiiina/YOLO-Colab
- https://clap0107.tistory.com/5
- 저거써볼거임

- 생각해보니 JSON 파일도 함께 사용해야함
- https://velog.io/@1-june/YOLOv5%EC%97%90%EA%B2%8C-%EC%83%88%EB%A1%9C%EC%9A%B4-%EA%B2%83-%EA%B0%80%EB%A5%B4%EC%B3%90%EB%B3%B4%EA%B8%B0
- https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#local-logging

In [ ]:
# 런타임 > 런타임 유형 변경> 하드웨어 가속기에서 GPU를 선택
# GPU 제대로 잡히는지 확인

### 전처리


#### 보도파손 이미지만 따로 저장

In [ ]:
def move_file_broken(destination_folder, json_data, json_file):
  num = 0

  for i in json_data['annotations']:
    if i['category_id'] == 'flatness_D' or i['category_id'] == 'flatness_E' or i['category_id'] == 'block_state_broken' or i['category_id'] == 'block_kind_bad':
        #print(json_file)
        #print()
        #print(os.path.join(jfolder, j))
        #print( os.path.join(dfolder, "image", "json_file"))

        # JSON 파일
        shutil.copyfile(os.path.join(jfolder, json_file), os.path.join(dfolder, "label_temp", json_file))
        # img 파일
        shutil.copyfile(os.path.join(ifolder, json_file[0:-4]+"jpg"), os.path.join(dfolder, "images", json_file[0:-4]+"jpg"))

        num = 1
        region_num = 0

  return num

In [ ]:
import shutil

jfolder = "/content/drive/MyDrive/yolov5/DATA/label_temp"
ifolder = "/content/drive/MyDrive/yolov5/DATA/data/images"

dfolder = "/content/drive/MyDrive/yolov5/DATA/brokenData"

n=1
m=0
for j in jsons:
  with open(os.path.join(jfolder, j)) as json_file:
    json_data = json.load(json_file)
    m = m + move_file_broken(dfolder, json_data, j)
    print(m, "/",n,"/",len(jsons))
    n+1

#### 라벨 to txt

txt 변환
- 참고
- https://velog.io/@1-june/YOLOv5%EC%97%90%EA%B2%8C-%EC%83%88%EB%A1%9C%EC%9A%B4-%EA%B2%83-%EA%B0%80%EB%A5%B4%EC%B3%90%EB%B3%B4%EA%B8%B0
- https://minmiin.tistory.com/14

In [ ]:
import re
import json
import os
import csv
import shutil

training_labels_folder = "/content/drive/MyDrive/yolov5/DATA/label_temp"
jsons = []

#for f in os.listdir(training_images_folder):
#  jsons.append(f[0:-3]+"json")

for f in os.listdir(training_labels_folder):
  jsons.append(f)

len(jsons)

In [ ]:
def save_as_yolo_format(destination_folder, json_data):
  num = 0
  img_width = json_data['images']['width'] # JSON 파일에서 이미지의 width 값이 있는 곳
  img_height = json_data['images']['height'] # JSON 파일에서 이미지의 height 값이 있는 곳

  yolov5_format_list = []
  for i in json_data['annotations']:
    if i['category_id'] == 'flatness_D' or i['category_id'] == 'flatness_E' or i['category_id'] == 'block_state_broken' or i['category_id'] == 'block_kind_bad':
        num = 1
        region_num = 0
        xmax = -1; xmin = 5000; ymax = -1; ymin = 5000
        for x, y in (i['segmentation']):
          if (x > xmax): xmax = x
          if (x < xmin): xmin = x
          if (y > ymax): ymax = y
          if (y < ymin): ymin = y

        xcentre = ((xmax + xmin) / 2) / json_data['images']['width']
        ycentre = ((ymax + ymin) / 2) / json_data['images']['height']
        bbox_width = (xmax - xmin) / json_data['images']['width']
        bbox_height = (ymax - ymin) / json_data['images']['height']
        yolov5_format = [region_num, xcentre, ycentre, bbox_width, bbox_height]
        yolov5_format_list.append(yolov5_format)

  file = open(os.path.join(destination_folder, re.sub(r'[^.]+$', 'txt', json_data['images']['file_name'])), 'w', newline='') # 파일명
  with file:
      write = csv.writer(file, delimiter= ' ')
      write.writerows(yolov5_format_list)
  return num

In [ ]:
destination_folder = "/content/drive/MyDrive/yolov5/DATA/data/labels"

n=1
m=0
for j in jsons:
  with open(os.path.join(training_labels_folder, j)) as json_file:
    json_data = json.load(json_file)
    m = m + save_as_yolo_format(destination_folder, json_data)
    print(m, "/",n,"/",len(jsons))
    n=n+1

##### 둘 다 합친 버전

In [ ]:
import re
import json
import os
import csv
import shutil

training_labels_folder = "/content/drive/MyDrive/yolov5/DATA/label_temp"
jsons = []

for f in os.listdir(training_labels_folder):
  jsons.append(f)

len(jsons)

In [ ]:
ifolder = "/content/drive/MyDrive/yolov5/DATA/data/images"
dfolder = "/content/drive/MyDrive/yolov5/DATA/brokenData"
jfolder = "/content/drive/MyDrive/yolov5/DATA/label_temp"

destination_folder = "/content/drive/MyDrive/yolov5/DATA/data/labels"

In [ ]:
def save_and_move(destination_folder, json_data, json_file):
  num = 0
  img_width = json_data['images']['width'] # JSON 파일에서 이미지의 width 값이 있는 곳
  img_height = json_data['images']['height'] # JSON 파일에서 이미지의 height 값이 있는 곳

  yolov5_format_list = []
  for i in json_data['annotations']:
    if i['category_id'] == 'flatness_D' or i['category_id'] == 'flatness_E' or i['category_id'] == 'block_state_broken' or i['category_id'] == 'block_kind_bad':
        num = 1
        region_num = 0
        xmax = -1; xmin = 5000; ymax = -1; ymin = 5000
        for x, y in (i['segmentation']):
          if (x > xmax): xmax = x
          if (x < xmin): xmin = x
          if (y > ymax): ymax = y
          if (y < ymin): ymin = y

        xcentre = ((xmax + xmin) / 2) / json_data['images']['width']
        ycentre = ((ymax + ymin) / 2) / json_data['images']['height']
        bbox_width = (xmax - xmin) / json_data['images']['width']
        bbox_height = (ymax - ymin) / json_data['images']['height']
        yolov5_format = [region_num, xcentre, ycentre, bbox_width, bbox_height]
        yolov5_format_list.append(yolov5_format)

        # JSON 파일
        shutil.copyfile(os.path.join(jfolder, json_file), os.path.join(dfolder, "label_temp", json_file))
        # img 파일
        shutil.copyfile(os.path.join(ifolder, json_file[0:-4]+"jpg"), os.path.join(dfolder, "images", json_file[0:-4]+"jpg"))

        file2 = open(os.path.join("/content/drive/MyDrive/yolov5/DATA/brokenData/labels", re.sub(r'[^.]+$', 'txt', json_data['images']['file_name'])), 'w', newline='') # 파일명
        with file2:
          write = csv.writer(file2, delimiter= ' ')
          write.writerows(yolov5_format_list)

  file = open(os.path.join(destination_folder, re.sub(r'[^.]+$', 'txt', json_data['images']['file_name'])), 'w', newline='') # 파일명
  with file:
      write = csv.writer(file, delimiter= ' ')
      write.writerows(yolov5_format_list)



  return num

In [ ]:
# 둘 다 합친 버전
ifolder = "/content/drive/MyDrive/yolov5/DATA/data/images"
dfolder = "/content/drive/MyDrive/yolov5/DATA/brokenData"
jfolder = "/content/drive/MyDrive/yolov5/DATA/label_temp"

destination_folder = "/content/drive/MyDrive/yolov5/DATA/data/labels"

n=1
m=0
for j in jsons:
  with open(os.path.join(training_labels_folder, j)) as json_file:
    json_data = json.load(json_file)
    m = m + save_and_move(destination_folder, json_data, j)
    print(m, "/",n,"/",len(jsons))
    n=n+1

In [ ]:
# 테스트코드

import json
import csv
import os
from PIL import Image, ImageDraw
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

img_path = "/content/drive/MyDrive/yolov5/DATA/brokenData/images/DC0109_2020-12-09 105819_00.jpg"
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

draw_img = Image.fromarray(img)
draw = ImageDraw.Draw(draw_img)

img_width = img.shape[1]
img_height = img.shape[0]
print(img_width, img_height)

with open("/content/drive/MyDrive/yolov5/DATA/brokenData/labels/DC0109_2020-12-09 105819_00.txt") as f:
    csv_reader = csv.reader(f, delimiter=' ')
    for row in csv_reader:
        xcentre = float(row[1]) * img_width
        ycentre = float(row[2]) * img_height
        bbox_width = float(row[3]) * img_width
        bbox_height = float(row[4]) * img_height
        x0 = xcentre - bbox_width/2
        x1 = xcentre + bbox_width/2
        y0 = ycentre - bbox_height/2
        y1 = ycentre + bbox_height/2

        draw.rectangle([x0, y0, x1, y1], outline='white', width=2)

plt.figure(figsize=(20, 10))
plt.imshow(draw_img)

#### train / val 나누기

In [ ]:
# 먼저 export파일 안에 있는 이미지들을 list로 만들어준다.
from glob import glob
img_list = glob('/content/drive/MyDrive/yolov5/DATA/brokenData/images/*.jpg')
print(len(img_list))

7153


In [ ]:
# 다음에 img_list를 train set과 valid set으로 나눠준다.

from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size = 0.2, random_state = 2000)
print(len(train_img_list), len(val_img_list))

5722 1431


In [ ]:
file = open("/content/drive/MyDrive/yolov5/DATA/train.txt", 'w', newline='') # 파일명
for i in train_img_list:
    file.write(i+"\n")
#for i in train_label_list:
#    file.write(i+"\n")
file.close()

In [ ]:
file = open("/content/drive/MyDrive/yolov5/DATA/valid.txt", 'w', newline='') # 파일명
for i in val_img_list:
    file.write(i+"\n")
#for i in val_label_list:
#    file.write(i+"\n")
file.close()

#### yaml 파일 생성

In [ ]:
%cd /content/drive/MyDrive/yolov5/DATA

/content/drive/MyDrive/yolov5/DATA


In [ ]:
import yaml

yaml_data = {"names":['broken'], # 클래스 이름
             "nc":1, # 클래스 수
             "path":"/content/drive/MyDrive/yolov5/DATA/data", # root 경로
             "train":("train.txt"), # train.txt 경로
             "val":("valid.txt"), # valid.txt 경로
             # "test":os.path.join(root_dir,"test.txt") # test.txt 경로
             }

with open(("custom.yaml"), "w") as f:
  yaml.dump(yaml_data, f)

### 실행

#### wandb 사용
- https://lynnshin.tistory.com/47 참고
- https://wandb.ai/site
- https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Intro_to_Weights_%26_Biases.ipynb
- https://docs.wandb.ai/guides/integrations/yolov5

In [ ]:
# 설치
!pip install wandb

#### YOLOv5 실행
- 각 파라메터 뜻 https://ropiens.tistory.com/44
- https://github.com/ultralytics/yolov5/blob/master/data/coco128.yaml
- https://sguys99.github.io/ds01

GPU 사용량
- https://github.com/ultralytics/yolov5/issues/8397

wandb 사용
- https://lynnshin.tistory.com/47#google_vignette

In [ ]:
%cd /content/drive/MyDrive/yolov5/YOLO/yolov5

In [ ]:
!python train.py \
  --img 416 \
  --batch 16 \
  --epochs 100 --save-period 1 \
  --data /content/drive/MyDrive/yolov5/DATA/custom.yaml \
  --cfg ./models/yolov5m.yaml \
  --weights /content/drive/MyDrive/yolov5/YOLO/yolov5/runs/train/results18/weights/best.pt \
  --name results \
  --bbox_interval 1

In [ ]:
# 테스트
!python detect.py --weights /content/drive/MyDrive/yolov5/YOLO/yolov5/runs/train/results12/weights/best.pt --source /content/drive/MyDrive/yolov5/DATA/test/20200106_1_86557.jpg

In [ ]:
# https://medium.com/@elvenkim1/how-to-convert-yolov5-to-tflite-523925d4704d
# 텐서플로우 라이트로 변환
!python export.py --weights /content/drive/MyDrive/yolov5/YOLO/yolov5/runs/train/results18/weights/best.pt --include tflite